In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer, precision_score

class LogitRFECV(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        C: float = 1.0,
        cv=TimeSeriesSplit(n_splits=5),
        scoring=precision_score,
        step: int = 1,
        random_state: int = 42
    ):
        """
        C: коэффициент регуляризации для LogisticRegression
        cv: cross-validation splitter
        scoring: функция метрики или строка
        step: количество признаков, удаляемых на каждой итерации
        random_state: для воспроизводимости
        """
        self.C = C
        self.cv = cv
        self.scoring = scoring
        self.step = step
        self.random_state = random_state
        self.selector_: RFECV = None

    def fit(self, X, y):
        estimator = LogisticRegression(            
            solver='liblinear',
            C=self.C,
            random_state=self.random_state
        )
        scorer = self.scoring
        if callable(self.scoring):
            scorer = make_scorer(self.scoring)

        self.selector_ = RFECV(
            estimator=estimator,
            step=self.step,
            cv=self.cv,
            scoring=scorer,
            n_jobs=-1
        )
        self.selector_.fit(X, y)
        return self

    def transform(self, X):
        if self.selector_ is None:
            raise ValueError("You must fit the selector before transforming.")
        return self.selector_.transform(X)

    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)


In [23]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score

# Генерируем пример данных
X, y = make_classification(
    n_samples=100, n_features=20, n_informative=5, n_redundant=2, random_state=42
)

# Создаём TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Создаём селектор
selector = LogitL1CVSelector(
    cs=np.logspace(-2, 1, 100),
    cv=tscv,
    scoring=precision_score,
    random_state=42
)

# Обучаем и сразу трансформируем
X_selected = selector.fit_transform(X, y)

# Вывод результатов
print("Лучший C:", selector.best_C_)
print("Лучший CV score:", selector.best_score_)
print("Выбранные признаки (индексы):", selector.selected_features_)
print("Форма X после отбора:", X_selected.shape)


/home/sergey/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sergey/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sergey/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sergey/.local/lib/python3.10

Лучший C: 0.43287612810830595
Лучший CV score: 0.7866666666666667
Выбранные признаки (индексы): [ 1  4  7 10 11 12]
Форма X после отбора: (100, 6)
